In [0]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need these
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow2/text_classification/imdb/main')

In [0]:
!pip install tensorflow-gpu==2.0.0-alpha0

     |████████████████████████████████| 332.1MB 56kB/s 
     |████████████████████████████████| 419kB 49.7MB/s 
     |████████████████████████████████| 3.0MB 53.8MB/s 
     |████████████████████████████████| 61kB 32.0MB/s 


In [0]:
import tensorflow as tf
import numpy as np
import pprint
import logging
import time

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 2.0.0-alpha0
GPU Enabled: True


In [0]:
def data_generator(f_paths, params):
  for f_path in f_paths:
    with open(f_path) as f:
      print('Reading', f_path)
      for line in f:
        line = line.rstrip()
        label, text = line.split('\t')
        text = text.split(' ')
        x = [params['word2idx'].get(w, len(word2idx)) for w in text]
        if len(x) >= params['max_len']:
          x = x[:params['max_len']]
        else:
          x += [0] * (params['max_len'] - len(x))
        y = int(label)
        yield x, y


def dataset(is_training, params):
  _shapes = ([params['max_len']], ())
  _types = (tf.int32, tf.int32)
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.batch(params['batch_size'])
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.batch(params['batch_size'])
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
class Model(tf.keras.Model):
  def __init__(self, params):
    super().__init__()
    
    self.embedding = tf.Variable(np.load('../vocab/word.npy'),
                                 dtype=tf.float32,
                                 name='pretrained_embedding',
                                 trainable=False,)
    
    self.drop1 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.drop2 = tf.keras.layers.Dropout(params['dropout_rate'])
    self.drop3 = tf.keras.layers.Dropout(params['dropout_rate'])
    
    self.rnn1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params['rnn_units'], return_sequences=True))
    self.rnn2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params['rnn_units'], return_sequences=True))
    self.rnn3 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(params['rnn_units'], return_sequences=True))
    
    self.drop_fc = tf.keras.layers.Dropout(params['dropout_rate'])
    self.fc = tf.keras.layers.Dense(2*params['rnn_units'], tf.nn.elu)
    
    self.out_linear = tf.keras.layers.Dense(2)
  
  
  def call(self, inputs, training=False):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)
    
    batch_sz = tf.shape(inputs)[0]
    rnn_units = 2*params['rnn_units']
    
    x = tf.nn.embedding_lookup(self.embedding, inputs)
    
    x = tf.reshape(x, (batch_sz*10*10, 10, 300))
    x = self.drop1(x, training=training)
    x = self.rnn1(x)
    x = tf.reduce_max(x, 1)
    
    x = tf.reshape(x, (batch_sz*10, 10, rnn_units))
    x = self.drop2(x, training=training)
    x = self.rnn2(x)
    x = tf.reduce_max(x, 1)
    
    x = tf.reshape(x, (batch_sz, 10, rnn_units))
    x = self.drop3(x, training=training)
    x = self.rnn3(x)
    x = tf.reduce_max(x, 1)
    
    x = self.drop_fc(x, training=training)
    x = self.fc(x)
    
    x = self.out_linear(x)
    
    return x

In [0]:
params = {
  'vocab_path': '../vocab/word.txt',
  'train_paths': [
    '../data/train_bt_part1.txt',
    '../data/train_bt_part2.txt',
    '../data/train_bt_part3.txt',
    '../data/train_bt_part4.txt',
    '../data/train_bt_part5.txt',
    '../data/train_bt_part6.txt',
  ],
  'test_paths': [
    '../data/test.txt',
  ],
  'num_samples': 25000*2,
  'num_labels': 2,
  'batch_size': 64,
  'max_len': 1000,
  'rnn_units': 200,
  'dropout_rate': 0.2,
  'clip_norm': 10.,
  'num_patience': 3,
  'lr': 3e-4,
}

In [0]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [0]:
word2idx = {}
with open(params['vocab_path']) as f:
  for i, line in enumerate(f):
    line = line.rstrip()
    word2idx[line] = i
params['word2idx'] = word2idx
params['vocab_size'] = len(word2idx) + 1


model = Model(params)
model.build(input_shape=(None, None))
pprint.pprint([(v.name, v.shape) for v in model.trainable_variables])

decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.95)
optim = tf.optimizers.Adam(params['lr'])
global_step = 0

history_acc = []
best_acc = .0

t0 = time.time()
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)


while True:
  # TRAINING
  for texts, labels in dataset(is_training=True, params=params):
    with tf.GradientTape() as tape:
      logits = model(texts, training=True)
      loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
      loss = tf.reduce_mean(loss)
  
    optim.lr.assign(decay_lr(global_step))
    grads = tape.gradient(loss, model.trainable_variables)
    grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    optim.apply_gradients(zip(grads, model.trainable_variables))

    if global_step % 50 == 0:
      logger.info("Step {} | Loss: {:.4f} | Spent: {:.1f} secs | LR: {:.6f}".format(
          global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
      t0 = time.time()
    global_step += 1

  # EVALUATION
  m = tf.keras.metrics.Accuracy()

  for texts, labels in dataset(is_training=False, params=params):
    logits = model(texts, training=False)
    y_pred = tf.argmax(logits, axis=-1)
    m.update_state(y_true=labels, y_pred=y_pred)
    
  acc = m.result().numpy()
  logger.info("Evaluation: Testing Accuracy: {:.3f}".format(acc))
  history_acc.append(acc)
  
  if acc > best_acc:
    best_acc = acc
    # you can save model here
  logger.info("Best Accuracy: {:.3f}".format(best_acc))
  
  if len(history_acc) > params['num_patience'] and is_descending(history_acc):
    logger.info("Testing Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    break

W0604 05:08:04.144721 139760335886208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f1c042d7f98>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0604 05:08:04.154609 139760335886208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f1c042e6828>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0604 05:08:04.162103 139760335886208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f1c042e6e80>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.
W0604 05:08:04.166756 139760335886208 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7f1c042a5668>: Note that this layer is not optimized for performance. Please use tf.keras.laye

[('bidirectional/forward_unified_lstm/kernel:0', TensorShape([300, 800])),
 ('bidirectional/forward_unified_lstm/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional/forward_unified_lstm/bias:0', TensorShape([800])),
 ('bidirectional/backward_unified_lstm/kernel:0', TensorShape([300, 800])),
 ('bidirectional/backward_unified_lstm/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional/backward_unified_lstm/bias:0', TensorShape([800])),
 ('bidirectional_1/forward_unified_lstm_1/kernel:0', TensorShape([400, 800])),
 ('bidirectional_1/forward_unified_lstm_1/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional_1/forward_unified_lstm_1/bias:0', TensorShape([800])),
 ('bidirectional_1/backward_unified_lstm_1/kernel:0', TensorShape([400, 800])),
 ('bidirectional_1/backward_unified_lstm_1/recurrent_kernel:0',
  TensorShape([200, 800])),
 ('bidirectional_1/backward_unified_lstm_1/bias:0', TensorShape([800])),
 ('bidirectional_2/forward_unified_lstm_2/kernel:0

I0604 05:08:27.043282 139760335886208 interactiveshell.py:2882] Step 0 | Loss: 0.6915 | Spent: 22.3 secs | LR: 0.000300
I0604 05:08:38.346872 139760335886208 interactiveshell.py:2882] Step 50 | Loss: 0.6155 | Spent: 11.3 secs | LR: 0.000299
I0604 05:08:49.525030 139760335886208 interactiveshell.py:2882] Step 100 | Loss: 0.6370 | Spent: 11.2 secs | LR: 0.000298
I0604 05:09:00.728225 139760335886208 interactiveshell.py:2882] Step 150 | Loss: 0.3955 | Spent: 11.2 secs | LR: 0.000298
I0604 05:09:11.964376 139760335886208 interactiveshell.py:2882] Step 200 | Loss: 0.4248 | Spent: 11.2 secs | LR: 0.000297
I0604 05:09:23.318131 139760335886208 interactiveshell.py:2882] Step 250 | Loss: 0.3979 | Spent: 11.4 secs | LR: 0.000296
I0604 05:09:34.908226 139760335886208 interactiveshell.py:2882] Step 300 | Loss: 0.2888 | Spent: 11.6 secs | LR: 0.000295
I0604 05:09:46.415040 139760335886208 interactiveshell.py:2882] Step 350 | Loss: 0.3135 | Spent: 11.5 secs | LR: 0.000295
I0604 05:09:57.880719 13976

Reading ../data/test.txt


I0604 05:12:52.996667 139760335886208 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.873
I0604 05:12:52.997944 139760335886208 interactiveshell.py:2882] Best Accuracy: 0.873


...

I0604 06:09:18.020703 139760335886208 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.913
I0604 06:09:18.021788 139760335886208 interactiveshell.py:2882] Best Accuracy: 0.917


Reading ../data/train_bt_part1.txt
Reading ../data/train_bt_part2.txt
Reading ../data/train_bt_part3.txt
Reading ../data/train_bt_part4.txt
Reading ../data/train_bt_part5.txt
Reading ../data/train_bt_part6.txt


I0604 06:09:40.311201 139760335886208 interactiveshell.py:2882] Step 10200 | Loss: 0.0696 | Spent: 113.3 secs | LR: 0.000178
I0604 06:09:51.865028 139760335886208 interactiveshell.py:2882] Step 10250 | Loss: 0.1023 | Spent: 11.6 secs | LR: 0.000177
I0604 06:10:03.506661 139760335886208 interactiveshell.py:2882] Step 10300 | Loss: 0.0439 | Spent: 11.6 secs | LR: 0.000177
I0604 06:10:15.313686 139760335886208 interactiveshell.py:2882] Step 10350 | Loss: 0.1518 | Spent: 11.8 secs | LR: 0.000176
I0604 06:10:26.762205 139760335886208 interactiveshell.py:2882] Step 10400 | Loss: 0.0862 | Spent: 11.4 secs | LR: 0.000176
I0604 06:10:38.290188 139760335886208 interactiveshell.py:2882] Step 10450 | Loss: 0.0407 | Spent: 11.5 secs | LR: 0.000176
I0604 06:10:49.670680 139760335886208 interactiveshell.py:2882] Step 10500 | Loss: 0.0511 | Spent: 11.4 secs | LR: 0.000175
I0604 06:11:01.067734 139760335886208 interactiveshell.py:2882] Step 10550 | Loss: 0.1261 | Spent: 11.4 secs | LR: 0.000175
I0604 0

Reading ../data/test.txt


I0604 06:13:59.575398 139760335886208 interactiveshell.py:2882] Evaluation: Testing Accuracy: 0.899
I0604 06:13:59.576631 139760335886208 interactiveshell.py:2882] Best Accuracy: 0.917
I0604 06:13:59.582616 139760335886208 interactiveshell.py:2882] Testing Accuracy not improved over 3 epochs, Early Stop
